***GENERATED CODE FOR whitepsaceclassificationdemo PIPELINE***
***DON'T EDIT THIS CODE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID0', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID0', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '1021', 'mean': '', 'stddev': '', 'min': '0026482c-a666-4d83-8387-1febcab99514', 'max': 'ff54b805-3ca5-4d22-999e-f75de7d0b71d', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'ChargeDetailID0', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID0')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'RVUModifier', 'transformation_label': 'String Indexer'}], 'feature': 'RVUModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '26.0', 'stddev': '0.0', 'min': '26', 'max': '26', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'RVUModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('RVUModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerName', 'transformation_label': 'String Indexer'}], 'feature': 'PayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': 'AARPMedicareComplete', 'max': 'WorkersComp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'PayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'WorkComp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'PayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'OtherGovernment', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'PayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Firstname38', 'transformation_label': 'String Indexer'}], 'feature': 'Firstname38', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': 'Agnieszka', 'max': 'Yu', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Firstname38', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Firstname38')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Lastname40', 'transformation_label': 'String Indexer'}], 'feature': 'Lastname40', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': 'Albert', 'max': 'Zuniga', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Lastname40', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Lastname40')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': 'Albert,Audra', 'max': 'Zuniga,Dorcas', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'ProviderCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': 'Albert,Audra-8F576202-A3BD-4943-9BA1-AA523E353890', 'max': 'Zuniga,Dorcas-E217C7D0-130E-47B6-AC58-59C99E52E0F6', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID56', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID56', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': '0026482c-a666-4d83-8387-1febcab99514', 'max': 'ff54b805-3ca5-4d22-999e-f75de7d0b71d', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'ChargeDetailID56', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID56')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SourceKey58', 'transformation_label': 'String Indexer'}], 'feature': 'SourceKey58', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': 'CO109', 'max': 'PR3', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'SourceKey58', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SourceKey58')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDate', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': '01/05/2020', 'max': '31/03/2020', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'ReasonDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityID63', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityID63', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': '087613d0-c7f3-4bcd-900a-abbc6ee3bdad', 'max': 'fde71f17-372e-4cde-95a7-d8b9b45a015a', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'FacilityID63', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityID63')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentID', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': '0a414089-7b3a-4378-ba12-23a2d7272bcf', 'max': 'fb68b47d-bad4-42c2-9ad0-9816864ea30a', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'DepartmentID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonCode', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '62.71', 'stddev': '82.27', 'min': '1', 'max': 'N807', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'ReasonCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': 'COB', 'max': 'TimelyFiling', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonDescription', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '1021', 'mean': '', 'stddev': '', 'min': 'Adjustmentbasedonthefindingsofarevieworganization/professionalconsult/manualadjudication/medicaladvisor/dentaladvisor/peerreview.',
                                                                                                                                                                                                                                                                                    'max': 'Yourclaimcontainsincompleteand/orinvalidinformation,andnoappealrightsareaffordedbecausetheclaimisunprocessable.Pleasesubmitanewclaimwiththecomplete/correctinformation.', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonDescription')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonType', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1021', 'mean': '', 'stddev': '', 'min': 'Co-Ins/Deduct', 'max': 'SPAY-PatientResponsibility', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'ReasonType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonType')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': sparkDF.columns}
    return data


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

%run whitepsaceclassificationdemoHooks.ipynb
try:
	sourcePreExecutionHook()

	whitepsaceclassificationdemodbfs = DBFSConnector.fetch(spark, "{'url': '/Whitespace/whitespace_Classification.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

	sourcePostExecutionHook(whitepsaceclassificationdemodbfs)

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
%run whitepsaceclassificationdemoHooks.ipynb
try:
	transformationPreExecutionHook()

	whitepsaceclassificationdemoautofe = TransformationMain.run(whitepsaceclassificationdemodbfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID0", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID0", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "0026482c-a666-4d83-8387-1febcab99514", "max": "ff54b805-3ca5-4d22-999e-f75de7d0b71d", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1021", "mean": "232.35", "stddev": "359.02", "min": "7.0", "max": "5154.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1021", "mean": "84.5", "stddev": "169.75", "min": "-203.35", "max": "2100.23", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CPTCodeID3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "1730.98", "stddev": "1185.84", "min": "1", "max": "4005", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DiagnosisCodeID2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "30401.86", "stddev": "23221.61", "min": "13501", "max": "94903", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DiagnosisCodeID1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "38832.84", "stddev": "21043.67", "min": "13501", "max": "94813", "missing": "0"}}, {"transformationsData": [{"feature_label": "RVUModifier", "transformation_label": "String Indexer"}], "feature": "RVUModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "26.0", "stddev": "0.0", "min": "26", "max": "26", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EnterpriseID18", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "1.0", "stddev": "0.06", "min": "1", "max": "2", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PatientNumber", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "204843.72", "stddev": "150965.6", "min": "52", "max": "472476", "missing": "0"}}, {"transformationsData": [{"feature_label": "PayerName", "transformation_label": "String Indexer"}], "feature": "PayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "AARPMedicareComplete", "max": "WorkersComp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "PayerCategory", "transformation_label": "String Indexer"}], "feature": "PayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "Aetna", "max": "WorkComp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsActive26", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SourceSystemID28", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}}, {"transformationsData": [{"feature_label": "PayerGroup", "transformation_label": "String Indexer"}], "feature": "PayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "Aetna", "max": "OtherGovernment", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "Firstname38", "transformation_label": "String Indexer"}], "feature": "Firstname38", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "Agnieszka", "max": "Yu", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "Lastname40", "transformation_label": "String Indexer"}], "feature": "Lastname40", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "Albert", "max": "Zuniga", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SpecialityID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "36.18", "stddev": "30.72", "min": "-1", "max": "120", "missing": "0"}}, {"transformationsData": [{"feature_label": "ProviderCategory", "transformation_label": "String Indexer"}], "feature": "ProviderCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "Albert,Audra", "max": "Zuniga,Dorcas", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsActive44", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}}, {"transformationsData": [{"feature_label": "ProviderName", "transformation_label": "String Indexer"}], "feature": "ProviderName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "Albert,Audra-8F576202-A3BD-4943-9BA1-AA523E353890", "max": "Zuniga,Dorcas-E217C7D0-130E-47B6-AC58-59C99E52E0F6", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SourceSystemID47", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TaxonomyID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "356.57", "stddev": "168.24", "min": "-1", "max": "770", "missing": "0"}}, {"transformationsData": [{"feature_label": "ChargeDetailID56", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID56", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "0026482c-a666-4d83-8387-1febcab99514", "max": "ff54b805-3ca5-4d22-999e-f75de7d0b71d", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TransactionReasonCodeID57", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "1170.3", "stddev": "591.9", "min": "10", "max": "3204", "missing": "0"}}, {"transformationsData": [{"feature_label": "SourceKey58", "transformation_label": "String Indexer"}], "feature": "SourceKey58", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "CO109", "max": "PR3", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount59", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1021", "mean": "232.35", "stddev": "359.02", "min": "7.0", "max": "5154.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "ReasonDate", "transformation_label": "String Indexer"}], "feature": "ReasonDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "01/05/2020", "max": "31/03/2020", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PracticeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "1.0", "stddev": "0.06", "min": "1", "max": "2", "missing": "0"}}, {"transformationsData": [{"feature_label": "FacilityID63", "transformation_label": "String Indexer"}], "feature": "FacilityID63", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "087613d0-c7f3-4bcd-900a-abbc6ee3bdad", "max": "fde71f17-372e-4cde-95a7-d8b9b45a015a", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CPTCodeID66", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "1730.98", "stddev": "1185.84", "min": "1", "max": "4005", "missing": "0"}}, {"transformationsData": [{"feature_label": "DepartmentID", "transformation_label": "String Indexer"}], "feature": "DepartmentID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "0a414089-7b3a-4378-ba12-23a2d7272bcf", "max": "fb68b47d-bad4-42c2-9ad0-9816864ea30a", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TransactionReasonCodeID70", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "1170.3", "stddev": "591.9", "min": "10", "max": "3204", "missing": "0"}}, {"transformationsData": [{"feature_label": "ReasonCode", "transformation_label": "String Indexer"}], "feature": "ReasonCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "62.71", "stddev": "82.27", "min": "1", "max": "N807", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "ReasonCategory", "transformation_label": "String Indexer"}], "feature": "ReasonCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "COB", "max": "TimelyFiling", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "ReasonDescription", "transformation_label": "String Indexer"}], "feature": "ReasonDescription", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "Adjustmentbasedonthefindingsofarevieworganization/professionalconsult/manualadjudication/medicaladvisor/dentaladvisor/peerreview.", "max": "Yourclaimcontainsincompleteand/orinvalidinformation,andnoappealrightsareaffordedbecausetheclaimisunprocessable.Pleasesubmitanewclaimwiththecomplete/correctinformation.", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "ReasonType", "transformation_label": "String Indexer"}], "feature": "ReasonType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1021", "mean": "", "stddev": "", "min": "Co-Ins/Deduct", "max": "SPAY-PatientResponsibility", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "isActive76", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "1.0", "stddev": "0.07", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1021", "mean": "0.52", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}}, {"feature": "ChargeDetailID0_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "330.2", "stddev": "250.68", "min": "0.0", "max": "805.0", "missing": "0"}}, {"feature": "RVUModifier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1021", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1021", "mean": "0.38", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}}, {"feature": "PayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "3.65", "stddev": "7.19", "min": "0.0", "max": "48.0", "missing": "0"}}, {"feature": "PayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "1.93", "stddev": "2.98", "min": "0.0", "max": "16.0", "missing": "0"}}, {"feature": "PayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "0.86", "stddev": "1.64", "min": "0.0", "max": "7.0", "missing": "0"}}, {"feature": "Firstname38_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "29.77", "stddev": "29.35", "min": "0.0", "max": "119.0", "missing": "0"}}, {"feature": "Lastname40_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "36.67", "stddev": "35.22", "min": "0.0", "max": "144.0", "missing": "0"}}, {"feature": "ProviderCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "38.34", "stddev": "36.91", "min": "0.0", "max": "149.0", "missing": "0"}}, {"feature": "ProviderName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "38.34", "stddev": "36.91", "min": "0.0", "max": "149.0", "missing": "0"}}, {"feature": "ChargeDetailID56_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "330.2", "stddev": "250.68", "min": "0.0", "max": "805.0", "missing": "0"}}, {"feature": "SourceKey58_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "6.59", "stddev": "11.17", "min": "0.0", "max": "68.0", "missing": "0"}}, {"feature": "ReasonDate_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "10.61", "stddev": "12.95", "min": "0.0", "max": "71.0", "missing": "0"}}, {"feature": "FacilityID63_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "17.09", "stddev": "18.17", "min": "0.0", "max": "82.0", "missing": "0"}}, {"feature": "DepartmentID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "1.82", "stddev": "2.93", "min": "0.0", "max": "21.0", "missing": "0"}}, {"feature": "ReasonCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "6.3", "stddev": "10.4", "min": "0.0", "max": "64.0", "missing": "0"}}, {"feature": "ReasonCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "1.69", "stddev": "2.49", "min": "0.0", "max": "16.0", "missing": "0"}}, {"feature": "ReasonDescription_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "6.3", "stddev": "10.4", "min": "0.0", "max": "64.0", "missing": "0"}}, {"feature": "ReasonType_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1021", "mean": "1.18", "stddev": "1.14", "min": "0.0", "max": "4.0", "missing": "0"}}]}))

	transformationPostExecutionHook(whitepsaceclassificationdemoautofe)

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
%run whitepsaceclassificationdemoHooks.ipynb
try:
	mlPreExecutionHook()

	dataAutoML=functionClassification(whitepsaceclassificationdemoautofe, ["Amount1", "Amount2", "CPTCodeID3", "DiagnosisCodeID2", "DiagnosisCodeID1", "EnterpriseID18", "PatientNumber", "IsActive26", "SourceSystemID28", "SpecialityID", "IsActive44", "SourceSystemID47", "TaxonomyID", "TransactionReasonCodeID57", "Amount59", "PracticeID", "CPTCodeID66", "TransactionReasonCodeID70", "isActive76", "ChargeDetailID0_stringindexer", "RVUModifier_stringindexer", "Gender_stringindexer", "PayerName_stringindexer", "PayerCategory_stringindexer", "PayerGroup_stringindexer", "Firstname38_stringindexer", "Lastname40_stringindexer", "ProviderCategory_stringindexer", "ProviderName_stringindexer", "ChargeDetailID56_stringindexer", "SourceKey58_stringindexer", "ReasonDate_stringindexer", "FacilityID63_stringindexer", "DepartmentID_stringindexer", "ReasonCode_stringindexer", "ReasonCategory_stringindexer", "ReasonDescription_stringindexer", "ReasonType_stringindexer"], "status")

	mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


**PREDICT ON TRAINED MODEL**

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

